In [1]:
import numpy as np
import pandas as pd
import EOS_extension
import TOVsolver
import MR_generator
import os
import time

In [2]:
EOS_og = pd.read_table('data/EOSCEFTVE1.dat', header=None).to_numpy()

keep = EOS_og[:,0] < 0.1601

EOS_start = EOS_og[keep,:]

In [4]:
def generate_tables(number_tables, EOS_start, nsamp_EOS, ext_type, MRL_size=100, maxm_thresh=1.8, newsavename=None):
    datapath = os.getcwd() + "/data/"
    if newsavename == None:
        EOSdir_name = ext_type+str(nsamp_EOS)+'EOS'
        MRLdir_name = ext_type+str(nsamp_EOS)+'MRL'
    else:
        EOSdir_name = ext_type+str(nsamp_EOS)+'EOS'+newsavename
        MRLdir_name = ext_type+str(nsamp_EOS)+'MRL'+newsavename
    
    if not(EOSdir_name in os.listdir(datapath)) and not(MRLdir_name in os.listdir(datapath)):
        os.makedirs(datapath+EOSdir_name)
        os.makedirs(datapath+MRLdir_name)
        
        
    numlist = []
    for file in os.listdir(datapath+EOSdir_name):
        if '.dat' in file:
            numlist.append(int(file[:-4]))
    
    if len(numlist) == 0:
        filenumstart = 0
    else:
        filenumstart = max(numlist) + 1

    i = 0
    done_25 = False
    done_50 = False
    done_75 = False
    num_failed = 0
    while i < number_tables:
        
        if ext_type == 'poly':
            EOS, ns, gammas, Ks = EOS_extension.extend(EOS_start, nsamp=nsamp_EOS,
                                                            ext_type=ext_type, max_gamma=9)
            param_string = "ns =" + str(ns) + ' gammas =' + str(gammas) + ' Ks =' + str(Ks)
        else:
            EOS, ns, cs = EOS_extension.extend(EOS_start, nsamp=nsamp_EOS, ext_type=ext_type)
            param_string = "ns =" + str(ns) + ' cs =' + str(cs)
        
        # extend EOS and store parameters

        MRL_table = TOVsolver.solve(EOS, MRL_size) #solve tov
        raw_mass = MRL_table[:,0]
        raw_radius = MRL_table[:,1]
        raw_Lambda = MRL_table[:,2]
        
        # create boolean arrays to test if points are good
        m2big = raw_mass < 4
        r2small = raw_radius > 7
        # the bool array of points we will keep
        keep = np.logical_and(m2big, r2small)
        # define new arrays we will keep
        radius = raw_radius[keep]
        mass = raw_mass[keep]
        Lambda = raw_Lambda[keep]
        
        #check if maximum mass is realistic
        maxm = np.max(raw_mass)
        if maxm < maxm_thresh:
#             print("maximum mass is too low")
            num_failed += 1
            i -= 1
        else:
            leng = len(radius) # get number of physical points
            MRL = np.zeros((leng, 3)) # initialize MRL table
            MRL[:,0], MRL[:,1], MRL[:,2] = mass, radius, Lambda # put into table

            EOSname = datapath + EOSdir_name + '/' + str(filenumstart + i) + '.dat' # make names for file
            MRLname = datapath + MRLdir_name + '/' + str(filenumstart + i) + '.dat'
            
#             print("saving file...")

            np.savetxt(EOSname, EOS, header=param_string) # save files
            np.savetxt(MRLname, MRL)
    
        
        if i*100/number_tables > 25 and done_25==False:
            print("The run is 25% complete")
            done_25=True
        elif i*100/number_tables > 50 and done_50==False:
            print("The run is 50% complete")
            done_50=True
        elif i*100/number_tables > 75 and done_75==False:
            print("The run is 75% complete")
            done_75=True
        
        i +=1
    print(str(num_failed)+' iterations failed for '+str(number_tables)+' successful tables.')

In [12]:
num_tables = 500
nsamp = 5
ext_type = 'cs'
newsavename = '_n0'

generate_tables(num_tables, EOS_start, nsamp, ext_type, newsavename=newsavename)

The run is 25% complete
The run is 50% complete
The run is 75% complete
512 iterations failed for 500 successful tables.


In [13]:
num_tables = 500
nsamp = 5
ext_type = 'cs'
newsavename = '_n0'

generate_tables(num_tables, EOS_start, nsamp, ext_type, newsavename=newsavename)

The run is 25% complete
The run is 50% complete
The run is 75% complete
496 iterations failed for 500 successful tables.


In [10]:
num_tables = 300
nsamp = 5
ext_type = 'cs'
newsavename = '_n0'

generate_tables(num_tables, EOS_start, nsamp, ext_type, newsavename=newsavename)

The run is 25% complete
The run is 50% complete
The run is 75% complete
336 iterations failed for 300 successful tables.


In [3]:
import TOVsolver
import pandas as pd
n0 = 0.16 #MeV/fm^3

n = EOS_start[:,0] 
p = EOS_start[:,1] 
e = EOS_start[:,2] 

# making cs=1 extention
max_n = 10*n0
n_step = 1e-3
size = int((max_n-n[-1])/n_step)

# initialize array
EOS_ex = np.zeros((size, 3))
# set starting values at n = n0
EOS_ex[0,0] = n[-1]
EOS_ex[0,1] = p[-1]
EOS_ex[0,2] = e[-1]

for k in range(size-1):
    # n_i+1
    EOS_ex[k+1,0] = EOS_ex[k,0] + n_step
    # p_i+1 (set c_s = 1)
    EOS_ex[k+1, 1] = EOS_ex[k, 1] + n_step * 1 * ((EOS_ex[k, 1] + EOS_ex[k, 2]) / EOS_ex[k, 0])
    # e_i+1
    EOS_ex[k+1,2] = EOS_ex[k,2] + n_step*((EOS_ex[k,1]+EOS_ex[k,2])/EOS_ex[k,0])

# get relevant sizes
size_smol = EOS_start.shape[0] -1 # -1 becuase we don't want last duplicated entry
size_ex = EOS_ex.shape[0]

# initialize array
cs_eq1_EOS = np.zeros((size_smol+size_ex, EOS_start.shape[1]))

cs_eq1_EOS[:size_smol,:] = EOS_start[:size_smol,:]
cs_eq1_EOS[size_smol:,:] = EOS_ex[:,:]

cs_eq1_MRL = TOVsolver.solve(cs_eq1_EOS, size=80)

np.savetxt(os.getcwd() + "/data/cs_eq1EOS_n0", cs_eq1_EOS)
np.savetxt(os.getcwd() + "/data/cs_eq1MRL_n0", cs_eq1_MRL)

In [14]:
maxm_n0 = np.max(cs_eq1_MRL[:,0])
print(maxm_n0)

4.086114839113669
